In [2]:
%pwd

'/Users/aakanksha/My_Codes/Chicken-Disease-Classfication-Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path    

In [4]:
from src.cnnClassifier.utils.common import read_yaml, create_directories
from src.cnnClassifier.constants import *

In [5]:

import os
import tensorflow as tf

In [6]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    def get_training_config(self)->TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )

        return training_config

In [10]:
import os
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),  # Convert PosixPath to string
            save_best_only=True
            )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
            ]

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [12]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2023-10-11 10:24:49,874: INFO: common: yaml file: config/config.yaml loaded suceessfully]
[2023-10-11 10:24:49,877: INFO: common: yaml file: params.yaml loaded suceessfully]
[2023-10-11 10:24:49,879: INFO: common: created directory at: artifacts]
[2023-10-11 10:24:49,880: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2023-10-11 10:24:49,881: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2023-10-11 10:24:49,883: INFO: common: created directory at: artifacts/training]


2023-10-11 10:24:49.969175: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-11 10:24:49.969273: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-11 10:24:49.969298: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-11 10:24:49.969882: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-11 10:24:49.970352: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[2023-10-11 10:24:52,695: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.SGD`.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/10


2023-10-11 10:24:53.920182: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2023-10-11 10:24:53.958731: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node SGD/AssignVariableOp.


19/19 [==============================] - 21s 857ms/step - loss: 12.6887 - accuracy: 0.5338 - val_loss: 13.4439 - val_accuracy: 0.6094
Epoch 2/10


/Users/aakanksha/miniforge3/envs/coffeeprogrammer/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


19/19 [==============================] - 12s 631ms/step - loss: 8.5387 - accuracy: 0.6149 - val_loss: 3.3269 - val_accuracy: 0.6719
Epoch 3/10
19/19 [==============================] - 12s 623ms/step - loss: 6.6865 - accuracy: 0.6182 - val_loss: 4.3894 - val_accuracy: 0.7031
Epoch 4/10
19/19 [==============================] - 12s 635ms/step - loss: 0.9435 - accuracy: 0.8818 - val_loss: 5.3873 - val_accuracy: 0.6406
Epoch 5/10
19/19 [==============================] - 12s 630ms/step - loss: 0.9099 - accuracy: 0.8784 - val_loss: 1.0505 - val_accuracy: 0.8906
Epoch 6/10
19/19 [==============================] - 14s 731ms/step - loss: 2.7526 - accuracy: 0.7804 - val_loss: 8.3579 - val_accuracy: 0.5312
Epoch 7/10
19/19 [==============================] - 13s 643ms/step - loss: 1.3281 - accuracy: 0.8581 - val_loss: 0.8569 - val_accuracy: 0.9219
Epoch 8/10
19/19 [==============================] - 14s 753ms/step - loss: 0.6855 - accuracy: 0.9088 - val_loss: 0.8073 - val_accuracy: 0.9375
Epoch 9/10